![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

In [22]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [23]:
# Definindo os caminhos dos arquivos
features_path = "features.txt"
activity_labels_path = "activity_labels.txt"
X_train_path = "train/X_train.txt"
y_train_path = "train/y_train.txt"
subject_train_path = "train/subject_train.txt"
X_test_path = "test/X_test.txt"
y_test_path = "test/y_test.txt"
subject_test_path = "test/subject_test.txt"

# Carregar nomes das features
features = pd.read_csv(features_path, delim_whitespace=True, header=None)
features = features[1].tolist()  # Extrair apenas os nomes das features

# Tornar os nomes das features únicos
features = pd.Series(features)
features_unique = features + "_" + features.groupby(features).cumcount().astype(str)
features_unique = features_unique.tolist()

# Carregar os rótulos das atividades
activity_labels = pd.read_csv(activity_labels_path, delim_whitespace=True, header=None, index_col=0)

# Carregar dados de treino
X_train = pd.read_csv(X_train_path, delim_whitespace=True, header=None, names=features_unique)
y_train = pd.read_csv(y_train_path, delim_whitespace=True, header=None)
subject_train = pd.read_csv(subject_train_path, delim_whitespace=True, header=None)

# Carregar dados de teste
X_test = pd.read_csv(X_test_path, delim_whitespace=True, header=None, names=features_unique)
y_test = pd.read_csv(y_test_path, delim_whitespace=True, header=None)
subject_test = pd.read_csv(subject_test_path, delim_whitespace=True, header=None)

# Mapear os rótulos para nomes de atividades
y_train = y_train[0].map(activity_labels[1])
y_test = y_test[0].map(activity_labels[1])




## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [24]:
%%time

# Instanciar o modelo de árvore de decisão
clf = DecisionTreeClassifier(ccp_alpha=0.001)

# Treinar o modelo
clf.fit(X_train, y_train)

# Prever nos conjuntos de treino e teste
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Avaliar a acurácia
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

(train_accuracy, test_accuracy)

CPU times: total: 2.02 s
Wall time: 6.04 s


(0.9757889009793254, 0.8805564981336953)

**Obs:** Conforme podemos observar acima, obtivemos uma diferença de aproximadamente 10% na acurácia entre os conjuntos de treino e teste. Isso sugere que o modelo está se ajustando excessivamente aos dados de treino, possivelmente aprendendo (ou "decorando") não apenas os padrões, mas também os ruídos presentes nesse conjunto. Esse comportamento indica que o modelo está sobreajustado (overfitted).

## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [28]:
%%time

# Aplicar PCA com apenas uma componente
pca = PCA(n_components=1)

# Reduzir as dimensões dos conjuntos de treino e teste
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Instanciar o modelo de árvore de decisão
clf_pca = DecisionTreeClassifier()

# Treinar o modelo
clf_pca.fit(X_train_pca, y_train)

# Prever nos conjuntos de treino e teste
y_train_pred_pca = clf_pca.predict(X_train_pca)
y_test_pred_pca = clf_pca.predict(X_test_pca)

# Avaliar a acurácia
train_accuracy_pca = accuracy_score(y_train, y_train_pred_pca)
test_accuracy_pca = accuracy_score(y_test, y_test_pred_pca)

(train_accuracy_pca, test_accuracy_pca)


CPU times: total: 125 ms
Wall time: 180 ms


(1.0, 0.41024770953512046)

**Obs:** Ao utilizar uma Análise de Componentes Principais (PCA) com apenas uma componente e aplicar uma árvore de decisão utilizando essa única variável explicativa, observamos uma acurácia de 100% no conjunto de treino e de aproximadamente 41% no conjunto de teste. Essa diferença significativa sugere que o modelo está completamente sobreajustado (overfitted) aos dados de treino. O modelo aprendeu perfeitamente os padrões presentes no conjunto de treino, mas não consegue generalizar para novos dados, resultando em uma baixa acurácia no conjunto de teste

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [30]:
%%time


# Lista de números de componentes a serem testados
n_components_list = [1, 2, 5, 10, 50]

# Armazenar resultados
results = []

# Configuração do GridSearch
param_grid = {'ccp_alpha': [0.001]}
clf = DecisionTreeClassifier()
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

for n_components in n_components_list:
    # Aplicar PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Treinar o modelo com GridSearch
    grid_search.fit(X_train_pca, y_train)
    
    # Melhor modelo encontrado
    best_clf = grid_search.best_estimator_
    
    # Previsão e avaliação de acurácia
    y_train_pred = best_clf.predict(X_train_pca)
    y_test_pred = best_clf.predict(X_test_pca)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    # Registrar resultados
    results.append({
        'n_components': n_components,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'best_params': grid_search.best_params_
    })

# Converter os resultados em DataFrame e exibir
results_df = pd.DataFrame(results)
print(results_df)

   n_components  train_accuracy  test_accuracy           best_params
0             1        0.499728       0.457075  {'ccp_alpha': 0.001}
1             2        0.612758       0.584662  {'ccp_alpha': 0.001}
2             5        0.846028       0.788599  {'ccp_alpha': 0.001}
3            10        0.892682       0.823889  {'ccp_alpha': 0.001}
4            50        0.919342       0.822871  {'ccp_alpha': 0.001}
CPU times: total: 2.05 s
Wall time: 5.32 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

### Análise dos Resultados

**O que aconteceu com a acurácia?**
- Conforme o número de componentes principais (`n_components`) aumenta, a acurácia tanto no conjunto de treino quanto no de teste também aumenta.
- Observamos que após um certo ponto (entre 10 e 50 componentes), a acurácia no conjunto de teste se estabiliza, sugerindo que adicionar mais componentes não melhora significativamente o desempenho do modelo.

**O que aconteceu com o tempo de processamento?**
- O tempo total de processamento aumentou com o uso de mais componentes principais. Isso é esperado, pois o modelo precisa processar mais variáveis, tornando o treinamento e a validação cruzada mais complexos e demorados.

### Escolha do Modelo

Para selecionar um modelo que não esteja sobreajustado (overfitted), consideramos:

1. **Diferença entre Acurácia de Treino e Teste**:
   - O modelo com 10 componentes principais (`n_components=10`) apresenta uma acurácia de treino de aproximadamente 89.3% e uma acurácia de teste de aproximadamente 82.4%.
   - A diferença entre as acurácias é de cerca de 6.9%, indicando que o modelo está generalizando bem sem estar excessivamente ajustado aos dados de treino.

2. **Desempenho Geral**:
   - O modelo com 50 componentes principais tem uma acurácia de teste muito semelhante ao modelo com 10 componentes, mas com um aumento na acurácia de treino, sugerindo um possível início de overfitting.

**Conclusão**:
- **Seleção do Modelo**: O modelo com **10 componentes principais (`n_components=10`)** é o mais balanceado entre a acurácia de treino e teste, sem sinais evidentes de overfitting. Portanto, ele seria a melhor escolha com base nos resultados apresentados.

- **Justificativa**: Este modelo oferece uma boa combinação de desempenho e generalização, mantendo a acurácia de teste alta enquanto minimiza a diferença entre as acurácias de treino e teste.


**Obs:** Agora, exploraremos mais dimensões dentro da melhor faixa encontrada anteriormente, em seguida observaremos se com as dimensionalidades escolhidas, poderemos obter melhores resultados entre o treino e teste.

In [33]:
%%time
# Lista de números de componentes a serem testados
n_components_list = [3, 7, 10, 15, 20, 30]

# Armazenar resultados
results = []

# Configuração do GridSearch
param_grid = {'ccp_alpha': [0.001]}
clf = DecisionTreeClassifier()
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

for n_components in n_components_list:
    # Aplicar PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Treinar o modelo com GridSearch
    grid_search.fit(X_train_pca, y_train)
    
    # Melhor modelo encontrado
    best_clf = grid_search.best_estimator_
    
    # Previsão e avaliação de acurácia
    y_train_pred = best_clf.predict(X_train_pca)
    y_test_pred = best_clf.predict(X_test_pca)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    # Registrar resultados
    results.append({
        'n_components': n_components,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'best_params': grid_search.best_params_
    })

# Converter os resultados em DataFrame e exibir
results_df = pd.DataFrame(results)
print(results_df)

   n_components  train_accuracy  test_accuracy           best_params
0             3        0.779516       0.754326  {'ccp_alpha': 0.001}
1             7        0.867247       0.777740  {'ccp_alpha': 0.001}
2            10        0.892682       0.824228  {'ccp_alpha': 0.001}
3            15        0.904788       0.814048  {'ccp_alpha': 0.001}
4            20        0.908460       0.819477  {'ccp_alpha': 0.001}
5            30        0.910909       0.823549  {'ccp_alpha': 0.001}
CPU times: total: 1.91 s
Wall time: 5.28 s


### Análise dos Resultados

**O que aconteceu com a acurácia?**
- **Treino**: A acurácia no conjunto de treino aumentou conforme o número de componentes principais foi ampliado, o que é esperado, já que mais componentes capturam mais variabilidade dos dados.
- **Teste**: A acurácia no conjunto de teste melhorou até atingir seu pico com 10 componentes principais (`n_components=10`), alcançando aproximadamente 82.4%. Após esse ponto, a acurácia variou levemente, mas não apresentou melhorias significativas, sugerindo que adicionar mais componentes não melhora a generalização do modelo.

**O que aconteceu com o tempo de processamento?**
- O tempo total de processamento foi de aproximadamente 5.28 segundos. O tempo de processamento aumentou conforme o número de componentes principais foi ampliado, o que é natural, pois mais dados precisaram ser processados.

### Escolha do Modelo

Com base nos resultados:

- **Melhor Acurácia de Teste**: O modelo com **10 componentes principais** (`n_components=10`) apresentou a melhor acurácia no conjunto de teste (82.4%) e uma acurácia de treino de 89.3%, sugerindo um bom equilíbrio entre desempenho e generalização.
  
- **Modelos com Acurácia Similar**: Os modelos com 20 e 30 componentes principais apresentaram acurácia de teste ligeiramente inferior ao modelo com 10 componentes, indicando que mais componentes não resultaram em melhor desempenho.

**Conclusão**:
- **Modelo Selecionado**: O modelo com **10 componentes principais** (`n_components=10`) continua sendo a melhor escolha, pois oferece o melhor equilíbrio entre acurácia de treino e teste, sem sinais evidentes de overfitting.
